# SQL 的五十道練習：初學者友善的資料庫入門

> 分組與聚合結果篩選

讀者如果是資料科學的初學者，可以略過下述的程式碼；讀者如果不是資料科學的初學者，欲使用 JupyterLab 執行本章節內容，必須先執行下述程式碼載入所需模組與連接資料庫。

In [1]:
%LOAD sqlite3 db=../databases/imdb.db timeout=2 shared_cache=true

In [2]:
ATTACH "../databases/nba.db" AS nba;

In [3]:
ATTACH "../databases/twElection2020.db" AS twElection2020;

## 複習一下

在第三章「從資料表選擇」我們提過使用 `DISTINCT` 保留字來為查詢的結果剔除重複值；第六章「排序查詢結果」我們提過在 SQL 敘述中加入 `ORDER BY` 指定欄位作為排序依據，預設為遞增排序。

In [4]:
SELECT DISTINCT director AS distinct_director
  FROM movies
 ORDER BY director
 LIMIT 10;

distinct_director
Aamir Khan
Adam Elliot
Akira Kurosawa
Alejandro G. Iñárritu
Alfred Hitchcock
Andrew Stanton
Anthony Russo
Asghar Farhadi
Billy Wilder
Bob Persichetti


## 以 `GROUP BY` 分組

對資料表中的欄位剔除重複值並且遞增排序，這樣的技巧在 SQL 與關聯式資料庫管理系統被稱為「分組」，在 SQL 敘述中加入 `GROUP BY` 就等同於 `DISTINCT` 與 `ORDER BY` 兩者同時作用的效果。

```sql
SELECT columns
  FROM table
 GROUP BY columns;
```

In [5]:
SELECT director
  FROM movies
 GROUP BY director
 LIMIT 10;

director
Aamir Khan
Adam Elliot
Akira Kurosawa
Alejandro G. Iñárritu
Alfred Hitchcock
Andrew Stanton
Anthony Russo
Asghar Farhadi
Billy Wilder
Bob Persichetti


我們也能夠指定多個資料表欄位作為分組依據，只需要在 `GROUP BY` 之後用逗號 `,` 隔開不同欄位名稱即可，這時候會產生笛卡兒積（Cartesian product）效果，意即不同欄位的獨一值會組成所有可能的集合，例如 `GROUP BY director, release_year` 會將導演與上映年份的所有可能組成顯示於查詢結果。

In [6]:
SELECT director,
       release_year
  FROM movies
 GROUP BY director,
          release_year
 LIMIT 10;

director,release_year
Aamir Khan,2007
Adam Elliot,2009
Akira Kurosawa,1950
Akira Kurosawa,1952
Akira Kurosawa,1954
Akira Kurosawa,1961
Akira Kurosawa,1963
Akira Kurosawa,1975
Akira Kurosawa,1985
Alejandro G. Iñárritu,2000


`GROUP BY` 除了能夠得到 `DISTINCT` 與 `ORDER BY` 同時作用的效果，另外一個重要功能是搭配聚合函數進行分組聚合的資料分析技巧。在第五章「函數」，我們將函數粗略分為兩大類：通用函數與聚合函數，其中「用來彙總資訊」的函數，稱為聚合函數（Aggregate functions）。於 `SELECT` 後使用聚合函數的時候，能夠將欄位資料彙總後輸出。

In [7]:
SELECT AVG(rating) AS avg_rating
  FROM movies;

avg_rating
8.30719999999998


舉例來說，現在我們希望計算不同年份 `release_year` 上映的電影平均評等，我們該怎麼做呢？比較直觀的想法是先知道有哪些年份。

In [8]:
SELECT release_year
  FROM movies
 GROUP BY release_year
 LIMIT 5;

release_year
1921
1924
1925
1926
1927


接著篩選不同上映年份的電影，計算這些電影的平均評等。

```sql
SELECT AVG(rating) AS avg_rating
  FROM players
 WHERE release_year = 1921;
SELECT AVG(rating) AS avg_rating
  FROM players
 WHERE release_year = 1924;
-- To be continued...
```

不過上映年份有為數眾多的獨一值，我們不太可能一一做資料表觀測值篩選然後計算平均評等。

In [9]:
SELECT COUNT(DISTINCT release_year) AS number_of_distinct_years
  FROM movies;

number_of_distinct_years
86


## 結合聚合函數與 `GROUP BY` 完成分組聚合

同時使用聚合函數（例如 `AVG()`、`COUNT()`、`SUM()`...等）以及 `GROUP BY` 可以便捷地達成分組聚合，完成上述計算不同年份 `release_year` 上映的電影平均評等。

```sql
SELECT AGGREGATE_FUNCTION(column) AS alias
  FROM table
 GROUP BY columns;
```

In [10]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year
 LIMIT 10;

release_year,avg_rating
1921,8.3
1924,8.2
1925,8.2
1926,8.2
1927,8.3
1928,8.2
1931,8.4
1934,8.1
1936,8.5
1939,8.13333333333333


## 以 `HAVING` 篩選分組聚合結果

在第七章「從資料表篩選」我們提過運用 `WHERE` 保留字搭配條件（Conditions）取出資料表符合「條件」的觀測值，這是一種作用於水平資料列「觀測值」的篩選方式。不過，假如我們希望針對分組聚合的結果進行篩選呢？

```sql
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 WHERE AVG(rating) >= 8.5
 GROUP BY release_year;
```

```
Error: sqlite3_statement_backend::prepare: misuse of aggregate: AVG()
```

這時我們得到了一個錯誤訊息：`misuse of aggregate: AVG()` 意即針對分組聚合的結果應用 `WHERE` 是不被允許的。在這樣的應用情境下，應該要改使用 `HAVING` 保留字加上帶有聚合函數的條件。`HAVING` 就像是分組聚合版本的 `WHERE`，兩者作用的維度不同，`WHERE` 篩選原始資料表列中的「觀測值」、`HAVING` 篩選聚合結果中的「欄位」。

```sql
SELECT AGGREGATE_FUNCTION(column) AS alias
  FROM table
 GROUP BY columns
HAVING conditions;
```

In [11]:
SELECT release_year,
       AVG(rating) AS avg_rating
  FROM movies
 GROUP BY release_year
HAVING AVG(rating) >= 8.5;

release_year,avg_rating
1936,8.5
1972,9.2
1974,8.6
1977,8.6
1994,8.8
1999,8.54
2002,8.5
2008,8.5


## 重點統整

- 在 SQL 敘述中加入 `GROUP BY` 就等同於 `DISTINCT` 與 `ORDER BY` 兩者同時作用的效果。
- `GROUP BY` 另外一個重要功能是搭配聚合函數進行分組聚合的資料分析技巧。
- 以 `HAVING` 篩選分組聚合結果。
- 這個章節學起來的 SQL 保留字：
    - `GROUP BY`
    - `HAVING`
- 將截至目前所學的 SQL 保留字集中在一個敘述中，寫作順序必須遵從標準 SQL 的規定。

```sql
SELECT DISTINCT columns AS alias,
       CASE WHEN condition_1 THEN result_1
            WHEN condition_2 THEN result_2
            ...
            ELSE result_n END AS alias
  FROM table
 WHERE conditions
 GROUP BY columns
HAVING conditions
 ORDER BY columns DESC
 LIMIT m;
```

## 練習題 25-29

練習題會涵蓋四個學習資料庫，記得要依據題目的需求，調整編輯器選單的學習資料庫，在自己電腦的 SQLiteStudio 寫出跟預期輸出相同的 SQL 敘述，寫作過程如果卡關了，可以參考附錄二「練習題參考解答」。

### 25. 從 `imdb` 資料庫的 `movies` 資料表計算每一年有幾部在 IMDb.com 獲得高評等的經典電影，參考下列的預期查詢結果。

註：在 `movies` 資料表中的所有電影都是高評等的經典電影，讀者不需要定義或篩選「高評等」。

預期輸出：(86, 2) 的查詢結果。

In [12]:
-- 礙於紙本篇幅僅顯示出前五列示意

release_year,number_of_movies
1921,1
1924,1
1925,1
1926,1
1927,1


### 26. 從 `imdb` 資料庫的 `movies` 資料表計算每一年有幾部在 IMDb.com 獲得高評等的經典電影，只顯示電影數在 5 部以上（`>= 5`）的年份，參考下列的預期查詢結果。

註：在 `movies` 資料表中的所有電影都是高評等的經典電影，讀者不需要定義或篩選「高評等」。

預期輸出：(17, 2) 的查詢結果。

release_year,number_of_movies
1957,6
1975,5
1994,5
1995,8
1997,5
1998,5
1999,5
2000,5
2001,5
2003,6


### 27. 從 `twElection2020` 資料庫的 `presidential` 資料表暸解台灣 2020 總統副總統的選舉結果，參考下列的預期查詢結果。

預期輸出：(3, 2) 的查詢結果。

candidate_id,total_votes
1,608590
2,5522119
3,8170231


### 28. 從 `nba` 資料庫的 `players` 資料表根據 `country` 暸解 NBA 由哪些國家的球員所組成，參考下列的預期查詢結果。

預期輸出：(42, 2) 的查詢結果。

country,number_of_players
USA,387
Canada,21
France,9
Germany,8
Australia,7
Spain,5
Serbia,5
Turkey,4
Nigeria,4
Slovenia,3


### 29. 從 `nba` 資料庫的 `players` 資料表根據 `country` 暸解 NBA 由哪些國家的球員所組成，只顯示球員數在 2 位以上（>= 2）並在 9 位以下（<=9）的國家，參考下列的預期查詢結果。

預期輸出：(24, 2) 的查詢結果。

country,number_of_players
France,9
Germany,8
Australia,7
Spain,5
Serbia,5
Turkey,4
Nigeria,4
Slovenia,3
Lithuania,3
Japan,3
